---
# Import Libraries
---

In [1]:
import numpy as np 
import pandas 
import csv
import warnings
warnings.filterwarnings("ignore")
src='/home/shared_data/textmining_genderrecognition_topicextraction/dataset_text_mining_Reuters_RCV2/'

---
# View dir dataset_text_mining_Reuters_RCV2/trc2
---

In [2]:
# path_tec2=src+'trc2/'

In [3]:
# src=path_tec2+'trc2.csv'
# ps=pandas.read_csv(src, encoding='latin-1')
# ps.head(2)

In [4]:
# src2=path_tec2+'headlines-docs.csv'
# ps2=pandas.read_csv(src2, encoding='latin-1')
# ps2.head(2)

In [5]:
# #verifica quanti record sono diversi tra le due tabelle
# x=ps==ps2
# len(np.where(x['message']==False)[0])

In [6]:
# src3=path_tec2+'headlines.txt'
# ps3=pandas.read_csv(src3, encoding='latin-1')
# ps3.head(2)

In [7]:
# src4=path_tec2+'headlines.csv'
# ps4=pandas.read_csv(src4, encoding='latin-1')
# ps4.head(2)

In [8]:
# src5=path_tec2+'TRC2-headlines-docs-TRECBLOG.v2'
# f = open(src5)     # This is a big file
# stampa_righe=0
# for line in f:                # Using 'for ... in' on file object
#     if stampa_righe==4:
#         break;
#     print (line)                # ',' keeps print from adding a line break
#     stampa_righe+=1
# f.close()

---
# View files on dataset_text_mining_Reuters_RCV2/rcv1
---

In [9]:
import os
from pathlib import Path
import time

def remove_tag(line,x,y):
    ris=line[x:(len(line)-y)]
    return ris

def convert_xml_to_dataframe(src):
    src_rcv=src
    f = open(src_rcv)     
    paragrafo=[]
    code=[]
    ris={}
    
    for line in f:                
        if(line[:3]=='<p>'):
            paragrafo.append(remove_tag(line,3,5))
        if(line[:7]=='<title>'):
            ris['title']=remove_tag(line,7,9)
        if(line[:10]=='<headline>'):
            ris['headline']=remove_tag(line,10,12)
        if(line[:8]=='<byline>'):
            ris['autore']=remove_tag(line,8,10)
        if(line[:14]=='  <code code="'):
            code.append(remove_tag(line,14,3))
            
    ris['code']=str(",".join(code))
    ris['text']=str(" ".join(paragrafo))
    f.close()
    ris_pd_xml=pandas.DataFrame([ris])
    return ris_pd_xml

def lista_file(src_dir):
    list_dir=os.listdir(src_dir)
    lista_file=[]
    for dir in list_dir:
        if( dir!='MD5SUMS' and dir!='dtds' and dir!='codes'):
            file=os.listdir(src_dir+dir)
            for f in file:
                if(f!='.ipynb_checkpoints'):
                    lista_file.append(dir+'/'+f)
    return lista_file

def build_df(src_dir,lista_file,rcv,indice_file='indice.txt',non_convertiti_file='file_non_convertiti.txt'):
    df=convert_xml_to_dataframe(src_dir+lista_file[0])
    i=1
    checkpoint=src+rcv
    indice_path= src+indice_file
    non_convertiti_path= src+non_convertiti_file
    non_convertiti=''
    
        
    if not Path(non_convertiti_path).exists():
        f= open(non_convertiti_path, "w")
        f.close()
    f=open(non_convertiti_path,"r")
    if( f.readline()==''):
        non_convertiti=''
        f.close()
    else:
        f=open(non_convertiti_path,"r")
        non_convertiti=str(f.read())
        f.close()
    
        
    if not Path(indice_path).exists():
        f= open(indice_path, "w")
        f.close()    
    f=open(indice_path,"r")
    if( f.readline()==''):
        ind=1
        f.close()
    else:
        f=open(indice_path,"r")
        ind=int(f.read())
        if Path(checkpoint).exists():
            df=pandas.read_csv(checkpoint)
        f.close()
    
    
    i=ind
    start =time.time()
    salva_ogni=25000
    for f in lista_file[ind:]:
        if( i%salva_ogni==0 or i%(len(lista_file)-1)==0):
            df.to_csv(checkpoint, index=False)
            indice= open(indice_path, "w")
            indice.truncate(0)
            indice.write(str(i))
            indice.close()
            end = time.time()
            t = time.strftime("%Hh:%Mm:%Ss",time.gmtime(int(end)-int(start)))
            print("elaborati",i,"file su",len(lista_file),",file corrotti:",(len(non_convertiti.split(','))-1),",la dim: del csv:",
                  len(df),",tempo impiegato per trasformare",salva_ogni,"file: "+t)
            start =time.time()
        i=i+1
        try:
            df2=convert_xml_to_dataframe(src_dir+f)    
            df=df.append(df2, ignore_index=True)
        except:
            non_convertiti=non_convertiti+','+f
            non_convertiti_l= open(non_convertiti_path, "w")
            non_convertiti_l.truncate(0)
            non_convertiti_l.write(str(non_convertiti))
            non_convertiti_l.close()
            print("file: ",non_convertiti_l)
    return df

def merge(ds1="rcv.csv",ds2="rcv_parte2.csv"):
    path_file=src+ds1
    df=pandas.read_csv(path_file)
    print("Originale:",len(df),'+')
    path_file2=src+ds2
    df2=pandas.read_csv(path_file2)
    print("Nuovo:",len(df2),'=')
    df=df.append(df2,ignore_index=True)
    print('----------------------')
    df.to_csv(path_file, index=False)    
    print('Risultato',len(df))
    os.remove(path_file2)


In [10]:
# src_dir=src+'rcv1/'
# lista_file=lista_file(src_dir)
# print("Totale file: ",len(lista_file))
# df=build_df(src_dir,lista_file,'rcv.csv')
# df

In [11]:
# src_dir=src+'rcv1/'
# lista_file=lista_file(src_dir)
# print("Totale file: ",len(lista_file))
# df=build_df(src_dir,lista_file,'rcv_parte2.csv')
# df

In [12]:
path_file=src+"rcv.csv"
if Path(src+"rcv_parte2.csv").exists():
    merge()
df=pandas.read_csv(path_file)
print(len(df))
df.head(2)

806801


,title,headline,autore,code,text
0,"UK: UK shares set for nervous week,upside seen...","UK shares set for nervous week,upside seen lim...",Dale Faulken,"UK,M11,MCAT",The UK share market is unlikely to make much h...
1,USA: CBOT wheat ends mostly lower on weather.,CBOT wheat ends mostly lower on weather.,NaN,"USA,M14,M141,MCAT",CBOT soft red winter wheat futures closed most...


---
# Code extraction
---

In [13]:
# estrai codes
def estrai_code(path,salta_,columns):
    dir_code=path
    f = open(dir_code)
    code={}
    salta=0
    for line in f:                
        if salta<salta_:
            salta+=1
        else:
            l=line.split()
            if(len(l)>0):
                code[l[0]]=' '.join(l[1:])
    f.close()
    topic_code=pandas.DataFrame.from_dict(code,orient='index').reset_index()
    topic_code.columns=columns
    return topic_code

path_c=src+'rcv1/codes/'
industry_codes=estrai_code(path_c+'industry_codes.txt',2,['industry_code','description'])
region_codes=estrai_code(path_c+'region_codes.txt',2,['region_code','description'])
topic_codes=estrai_code(path_c+'topic_codes.txt',2,['topic_code','description'])

In [14]:
if not Path(src+"industry_codes.csv").exists():
    industry_codes.to_csv(src+"industry_codes.csv", index=False)
if not Path(src+"region_codes.csv").exists():
    region_codes.to_csv(src+"region_codes.csv", index=False)
if not Path(src+"topic_codes.csv").exists():
    topic_codes.to_csv(src+"topic_codes.csv", index=False)

---
# Extraction multilingual data
---

In [15]:
def convert_xml_to_dataframe(src):
    src_rcv=src
    f = open(src_rcv)     
    paragrafo=[]
    code=[]
    ris={}
    
    for line in f:                
        if(line[:3]=='<p>'):
            paragrafo.append(remove_tag(line,3,5))
        if(line[:7]=='<title>'):
            ris['title']=remove_tag(line,7,9)
        if(line[:10]=='<headline>'):
            ris['headline']=remove_tag(line,10,12)
        if(line[:8]=='<byline>'):
            ris['autore']=remove_tag(line,8,10)
        if(line[:12]=='<code code="'):
            code.append(remove_tag(line,12,3))
            
    ris['code']=str(",".join(code))
    t=str(" ".join(paragrafo))
    ris['text']=t[:len(t)-7]
    f.close()
    ris_pd_xml=pandas.DataFrame([ris])
    return ris_pd_xml

def lista_file_multilingual_(src_dir):
    list_dir=os.listdir(src_dir)
    lista_file=[]
    i=0
    ind_split=[]
    for dir in list_dir:
        if(dir!='readme.txt' and dir!='MD5SUMS' and dir!='.ipynb_checkpoints'):
            dir_multilingual=os.listdir(src_dir+dir)
            ind_split.append(i)
            print(i)
            for dir_region in dir_multilingual:
                if(dir_region!='.ipynb_checkpoints'):
                    file=os.listdir(src_dir+dir+'/'+dir_region)
                    for f in file:
                        if(f!='.ipynb_checkpoints'):
                            lista_file.append(dir+'/'+dir_region+'/'+f)
                            i+=1
    return lista_file

In [16]:
# src_dir_multilingual=src+'RCV2_Multilingual_Corpus/'
# lista_file_multilingual=lista_file_multilingual_(src_dir_multilingual)
# print("Totale file: ",len(lista_file_multilingual))
# df_multilingual=build_df(src_dir_multilingual,lista_file_multilingual,'rcv_multilingual.csv',
#     indice_file='indice_multilingual.txt',non_convertiti_file='file_non_convertiti_multilingual.txt')
# df_multilingual

In [17]:
# src_dir_multilingual=src+'RCV2_Multilingual_Corpus/'
# lista_file_multilingual=lista_file_multilingual_(src_dir_multilingual)
# print("Totale file: ",len(lista_file_multilingual))
# df_multilingual=build_df(src_dir_multilingual,lista_file_multilingual,'rcv_multilingual_parte2.csv',
#     indice_file='indice_multilingual.txt',non_convertiti_file='file_non_convertiti_multilingual.txt')
# df_multilingual

In [18]:
path_file_multilingual=src+"rcv_multilingual.csv"
if Path(src+"rcv_multilingual_parte2.csv").exists():
    merge(ds1='rcv_multilingual.csv',ds2='rcv_multilingual_parte2.csv')
df_multilingual=pandas.read_csv(path_file_multilingual)
print(len(df))
df_multilingual.head(2)

806801


,title,headline,autore,code,text
0,NaN,[台灣央行]標售50億台幣1個月期NCDs加權平均得標利率為6.385%,NaN,"ASIAZ,DEVGCOZ,EASIAZ,TAIWAN,M13,MCAT",〔路透社台北24日電〕 台灣央行週四午後標售50億台幣的1個月期可轉讓定期存單(NCDs...
1,NaN,美國經濟學家預期今年民間借款需求相當不錯,NaN,"NAMZ,USA,USAZ,USW,E12,ECAT","〔路透社紐約29日電〕 美國著名經濟學家考夫曼表示,美國經濟成長強勁,加上短期利率僅會小..."
